# Generating Event chains in Events format
* Since doing this at run time takes very long (looping through the entire event list)
* Run after `data_generation/generate_day_trips.ipynb` has generated the chains.

In [2]:
from Environment.DataStructures.Event import Event
from Environment.DataStructures.Bus import Bus
from Environment.DataStructures.Stop import Stop
from Environment.enums import BusStatus, BusType, EventType, MCTSType
from src.utils import *
from tqdm import tqdm
import numpy as np
import pandas as pd
import pickle
import os
import json

In [9]:
a = {'b':{}}
print(a)
del a['b']
a

{'b': {}}


{}

In [3]:
def load_initial_state(bus_plan, trip_plan, random_seed=100):
    print("Loading initial states...")
    active_stops = []

    Buses = {}
    for bus_id, bus_info in bus_plan.items():
        bus_type = bus_info['service_type']
        if bus_type == 'regular':
            bus_type = BusType.REGULAR
        else:
            bus_type = BusType.OVERLOAD

        bus_status = BusStatus.IDLE
        bus_capacity = bus_info['vehicle_capacity']
        bus_starting_depot = bus_info['starting_depot']
        bus_block_trips = np.asarray(bus_info['trips'])

        bus_block_trips = [tuple(bus_block_trip) for bus_block_trip in bus_block_trips]
        for i, bus_block_trip in enumerate(bus_block_trips):
            block_id = bus_block_trip[0]
            trip_id = bus_block_trip[1]
            trip_info = trip_plan[trip_id]
            stop_id_original = trip_info['stop_id_original']
            active_stops.extend(stop_id_original)
            if i == 0:
                st = trip_plan[trip_id]['scheduled_time']
                st = [str_timestamp_to_datetime(st).time().strftime('%H:%M:%S') for st in st][0]
                # Add when the bus should reach next stop as state change
                t_state_change = str_timestamp_to_datetime(f"{starting_date_str} {st}")

        bus = Bus(bus_id,
                  bus_type,
                  bus_status,
                  bus_capacity,
                  bus_block_trips)
        bus.current_stop = bus_starting_depot
        bus.current_load = 0
        bus.t_state_change = t_state_change
        Buses[bus_id] = bus

    Stops = {}
    for active_stop in active_stops:
        stop = Stop(stop_id=active_stop)
        Stops[active_stop] = stop

    print(f"Added {len(Buses)} buses and {len(Stops)} stops.")
    return Buses, Stops


def load_events(starting_date, Buses, Stops, trip_plan, sampled_travel_time, random_seed=100, chain=1):
    # print("Adding events...")
    np.random.seed(random_seed)
    has_broken = False
    is_weekend = 0 if dt.datetime.strptime(starting_date, '%Y-%m-%d').weekday() < 5 else 1

    # Initial events
    # Includes: Trip starts, passenger sampling
    # all active stops that buses will pass
    events = []

    event_file = 'events_all_vehicles_chain_{chain}.pkl'
    # event_file = 'events_2_vehicles.pkl'
    saved_events = f'scenarios/baseline/data/{event_file}'

    pbar = tqdm(Buses.items())
    for bus_id, bus in pbar:
        if bus.type == BusType.OVERLOAD:
            continue
        blocks_trips = bus.bus_block_trips

        # Start trip (assuming trips are in sequential order)
        block = blocks_trips[0][0]
        trip = blocks_trips[0][1]
        st = trip_plan[trip]['scheduled_time']
        st = [str_timestamp_to_datetime(st).time().strftime('%H:%M:%S') for st in st][0]
        event_datetime = str_timestamp_to_datetime(f"{starting_date_str} {st}")
        event = Event(event_type=EventType.VEHICLE_START_TRIP,
                        time=event_datetime,
                        type_specific_information={'bus_id': bus_id})
        events.append(event)

        # Populate stops
        for block_trip in blocks_trips:
            block = int(block_trip[0])
            trip = block_trip[1]
            route_id = trip_plan[trip]['route_id']
            route_direction = trip_plan[trip]['route_direction']
            route_id_dir = f"{route_id}_{route_direction}"
            scheduled_time = trip_plan[trip]['scheduled_time']
            stop_id_original = trip_plan[trip]['stop_id_original']
            scheduled_time = [str_timestamp_to_datetime(st).strftime('%Y-%m-%d %H:%M:%S') for st in scheduled_time]

            for stop_sequence in range(len(scheduled_time)):
                # sampled_travel_time['23_FROM DOWNTOWN', 2310, 32, 'DWMRT', pd.Timestamp('2021-08-23 05:41:00')]
                val = sampled_travel_time[route_id_dir, block, stop_sequence + 1, stop_id_original[stop_sequence], pd.Timestamp(scheduled_time[stop_sequence])]
                load = val['sampled_loads']
                ons = val['ons']
                offs = val['offs']

                pbar.set_description(
                    f"Processing {block}, {stop_id_original[stop_sequence]}, {scheduled_time[stop_sequence]}, {route_id_dir}, {load}, {ons}, {offs}")
                # making sure passengers arrives before the bus
                event_datetime = str_timestamp_to_datetime(f"{scheduled_time[stop_sequence]}") - dt.timedelta(minutes=EARLY_PASSENGER_DELTA_MIN)

                event = Event(event_type=EventType.PASSENGER_ARRIVE_STOP,
                                time=event_datetime,
                                type_specific_information={'route_id_dir': route_id_dir,
                                                            'block_abbr': block,
                                                            'stop_sequence': stop_sequence + 1,
                                                            'stop_id': stop_id_original[stop_sequence],
                                                            'load': load, 'ons': ons, 'offs': offs})
                events.append(event)

                # people will leave after N minutes.
                event = Event(event_type=EventType.PASSENGER_LEAVE_STOP,
                                time=event_datetime + dt.timedelta(minutes=PASSENGER_TIME_TO_LEAVE),
                                type_specific_information={'route_id_dir': route_id_dir,
                                                            'stop_id': stop_id_original[stop_sequence],
                                                            'time': event_datetime})
                events.append(event)

    events.sort(key=lambda x: x.time, reverse=False)
    
    return events

In [4]:
config_path = 'scenarios/baseline/data/config.json'
with open(config_path) as f:
    config = json.load(f)

config_path = f'scenarios/baseline/data/{config["trip_plan"]}'
with open(config_path) as f:
    trip_plan = json.load(f)

config_path = f'scenarios/baseline/data/{config["vehicle_plan"]}'
with open(config_path) as f:
    bus_plan = json.load(f)

starting_date_str = '2021-08-23'
starting_date = dt.datetime.strptime(starting_date_str, '%Y-%m-%d')
starting_time = dt.time(0, 0, 0)
starting_datetime = dt.datetime.combine(starting_date, starting_time)
Buses, Stops = load_initial_state(bus_plan, trip_plan)

Loading initial states...
Added 12 buses and 394 stops.


In [5]:
chain_dir = 'scenarios/baseline/chains'
chain_count = 4

chains = []
for chain in range(chain_count):
    event_chain = []
    with open(f'{chain_dir}/ons_offs_dict_chain_{chain + 1}.pkl', 'rb') as handle:
        sampled_ons_offs_dict = pickle.load(handle)
    for k, v in sampled_ons_offs_dict.items():
        print(v)
        break
    passenger_events = load_events(starting_date_str, Buses, Stops, trip_plan, sampled_ons_offs_dict, random_seed=100, chain=chain)
    chains.append(passenger_events)
    
    save_to = f'{chain_dir}/ons_offs_dict_chain_{chain + 1}_processed.pkl'
    
    with open(save_to, 'wb') as handle:
        pickle.dump(passenger_events, handle, protocol=pickle.HIGHEST_PROTOCOL)

{'sampled_loads': 8, 'ons': 8.0, 'offs': 0.0}


Processing 2503, 25ACLASN, 2021-08-23 18:36:00, 25_NORTHBOUND, 0, 0.0, 0.0: 100%|██████████| 12/12 [00:01<00:00,  8.04it/s]    


{'sampled_loads': 6, 'ons': 6.0, 'offs': 0.0}


Processing 2503, 25ACLASN, 2021-08-23 18:36:00, 25_NORTHBOUND, 4, 0.0, 4.0: 100%|██████████| 12/12 [00:01<00:00,  7.93it/s]    


{'sampled_loads': 8, 'ons': 8.0, 'offs': 0.0}


Processing 2503, 25ACLASN, 2021-08-23 18:36:00, 25_NORTHBOUND, 0, 0.0, 0.0: 100%|██████████| 12/12 [00:01<00:00,  7.81it/s]    


{'sampled_loads': 9, 'ons': 9.0, 'offs': 0.0}


Processing 2503, 25ACLASN, 2021-08-23 18:36:00, 25_NORTHBOUND, 3, 0.0, 3.0: 100%|██████████| 12/12 [00:01<00:00,  8.04it/s]    


even if we get them TMC hcannel ids are arbitrary
map matching is a very old art

ask can you give us road GPS coords/linestrings of their network
plot on kepler

we will do it ourselves.
go foward with NDA if needed

In [23]:
len(passenger_events)

84806

In [18]:
sum(1 for pe in passenger_events if pe.event_type == EventType.VEHICLE_START_TRIP)

100

In [19]:
sum(1 for pe in passenger_events if pe.event_type == EventType.PASSENGER_ARRIVE_STOP)

42353

In [32]:
for chain in chains:
    print(chain[100])

2021-08-23 04:51:58,PASSENGER_ARRIVE_STOP,{'route_id_dir': '23_FROM DOWNTOWN', 'block_abbr': 2311, 'stop_sequence': 17, 'stop_id': 'CRERICWN', 'load': 5, 'ons': 0.0, 'offs': 5.0}
2021-08-23 04:51:58,PASSENGER_ARRIVE_STOP,{'route_id_dir': '23_FROM DOWNTOWN', 'block_abbr': 2311, 'stop_sequence': 17, 'stop_id': 'CRERICWN', 'load': 1, 'ons': 1.0, 'offs': 0.0}
2021-08-23 04:51:58,PASSENGER_ARRIVE_STOP,{'route_id_dir': '23_FROM DOWNTOWN', 'block_abbr': 2311, 'stop_sequence': 17, 'stop_id': 'CRERICWN', 'load': 2, 'ons': 1.0, 'offs': 0.0}
2021-08-23 04:51:58,PASSENGER_ARRIVE_STOP,{'route_id_dir': '23_FROM DOWNTOWN', 'block_abbr': 2311, 'stop_sequence': 17, 'stop_id': 'CRERICWN', 'load': 0, 'ons': 1.0, 'offs': 0.0}


In [34]:
import time
np.random.shuffle(chains[0])
start = time.time()

chains[0].sort(key=lambda x: x.time, reverse=False)
end = time.time() - start
print(end)


0.06398129463195801


In [47]:
a = [pe for pe in chains[0] if (pe.event_type != EventType.PASSENGER_ARRIVE_STOP) and (pe.event_type != EventType.PASSENGER_LEAVE_STOP)]
b = [pe for pe in chains[1] if (pe.event_type == EventType.PASSENGER_ARRIVE_STOP) or (pe.event_type == EventType.PASSENGER_LEAVE_STOP)]
c = a + b
c.sort(key=lambda x: x.time, reverse=False)
len(c)

84806